# Behavioral profile stratification via unsupervised learning

In [1]:
from dataset import access_db, demographics, data_path
from features import *

Created directory /odf-data: 2019-07-04-12-53-56



* Access odf-lab database 

In [2]:
subj_list, p_enc, df_dict = access_db()

In [3]:
demographics(subj_list, p_enc)

Period span: 01/05/2013 -- 31/10/2018

N of subjects: 205

Average number of assessments: 5.522
Median number of assessments: 5.0
Maximum number of assessments: 24
Minimum number of assessments: 1

Average number of encounters: 1.439
Median number of assessments: 1.0
Maximum number of assessments: 5
Minimum number of assessments: 1

Instrument list:
wisc-iii
ados-2modulo2
wppsi-iiifascia40-73
wisc-iv
ados-2modulotoddler
srs
psi-sf
wppsi
wais-iv
ados-2modulo1
emotionalavailabilityscales
leiterinternationalperformancescale-revised
ados-2modulo3
griffithsmentaldevelopmentscales
wppsi-iiifascia26-311
vineland-ii
N of selected instruments: 16


Mean age of the subjects: 11.043261101002557 -- Standard deviation: 5.097775697784719
N Female: 34 -- N Male: 171



In [4]:
behr = create_tokens(df_dict)

Average length of behavioral sequences: 5.522



In [5]:
lev1 = behr_level1(behr)

In [6]:
lev2 = behr_level2(behr)

In [7]:
lev3 = behr_level3(behr)

In [8]:
lev4 = behr_level4(behr)

In [9]:
v1, _ = create_vocabulary(lev1, level=1)
v2, _ = create_vocabulary(lev2, level=2)
v3, _ = create_vocabulary(lev3, level=3)
v4, _ = create_vocabulary(lev4, level=4)

Vocabulary size:1349

Vocabulary size:1198

Vocabulary size:514

Vocabulary size:1167



In [10]:
out_behr_lev1 = create_behr(lev1, v1, level=1)
out_behr_lev2 = create_behr(lev2, v2, level=2)
out_behr_lev3 = create_behr(lev3, v3, level=3)
out_behr_lev4 = create_behr(lev4, v4, level=4)